In [1]:
import numpy as np
from collections import defaultdict
import math
from scipy import stats

## Week 1: what is the probability of a sequence Pr(EA...AE)

Slide 20 of Week 1

In [2]:
def get_total_probability(transition_matrix, initial_probabilities, sequence):
    total_probability = initial[sequence[0]]
    
    for i in range(0, len(sequence)-1, 1):
        frm = sequence[i]
        to = sequence[i+1]
        total_probability *= transition_matrix[frm, to]
    
    return total_probability

Input data here

In [3]:
transition = np.array([[0.3, 0.7], [0.4, 0.6]])
initial = np.array([0.2, 0.8])
sequence = [0, 1, 1, 0, 0, 1]

get_total_probability(transition, initial, sequence)

0.007055999999999999

## Week 1: fitting a markov chain using maximum likelihood estimate

Slide 22 of Week 1

p[i, j] = # transitions from s[i] to s[j] / # transitions from s[i]

In [4]:
def fit_markov_chain(transition_matrix, sequence):
    transition_counts = np.zeros(transition_matrix.shape)
    from_counts = np.zeros(transition_matrix.shape[0])
    
    for i in range(0, len(sequence)-1, 1):
        frm = sequence[i]
        to = sequence[i+1]
        transition_counts[frm, to] += 1
        from_counts[frm] += 1
    
    print("Transition counts\n", transition_counts)
    print("Transition from counts\n", from_counts)
    
    for i in range(transition_matrix.shape[0]):
        transition_counts[i, :] /= from_counts[i]
    
    return transition_counts

Input data here

In [5]:
transition = np.array([[0.3, 0.7], [0.4, 0.6]])
sequence = [0, 1, 1, 0, 0, 1]

print("\nMax likelihood estimate\n", fit_markov_chain(transition, sequence))

Transition counts
 [[1. 2.]
 [1. 1.]]
Transition from counts
 [3. 2.]

Max likelihood estimate
 [[0.33333333 0.66666667]
 [0.5        0.5       ]]


## Week 1: PageRank

Slide 39 of Week 1

Adjust the value of power if it takes too long or does not converge

In [6]:
def page_rank(graph, d, power):
    N = graph.shape[0]
    graph = graph/graph.sum(axis = 1, keepdims = True)
    M = (1 - d) * (np.ones(N) / N) + d * graph
    
    return np.linalg.matrix_power(M, power)[0, :]

Input data here

In [7]:
L = np.array([[0, 1, 1], [1, 0, 0], [0, 1, 0]])
d = 0.85

page_rank(L, d, 1000)

array([0.38778971, 0.39739966, 0.21481063])

## Week 2: Alpha

In his extra notes on alphas and betas under week 2, this calculates everything

In [8]:
def pretty_print_dot(a, b):
    joined_ab = ["(" + str(x) + " * " + str(y) + ")" for x, y in zip(a, b)]
    return " + ".join(joined_ab)

In [9]:
def calculate_alphas(M, p, B, v):
    N, T = p.shape[0], v.shape[0]
    alphas = np.zeros(shape=(N, T))
    
    print("t1")
    for i in range(len(p)):
        alphas[i, 0] = p[i] * B[i][v[0]]
        print("alpha", i+1, "=", p[i], "*", B[i][v[0]], "=", alphas[i, 0])

    def recursive_step(t):
        if t != len(v):
            print("t" + str(t+1))
            for i in range(N):
                alphas[i, t] = B[i][v[t]] * np.dot(alphas[:, t - 1], M[:, i])
                print("alpha", i+1, "=", B[i][v[t]], "* [", pretty_print_dot(alphas[:, t - 1], M[:, i]), "] =", alphas[i, t])

            recursive_step(t + 1)
        
    recursive_step(1)
    return alphas

In [10]:
def get_h_plus_one(state, alphas, M, v, prints = True):
    N = M.shape[1]
    h_distribution = [alphas[i, -1]/sum(alphas[:, -1]) for i in range(N)]
    
    prob = 0
    calc = []
    
    for i in range(N):
        calc.append(str(h_distribution[i]) + " * " + str(M[i, state]))
        prob += h_distribution[i] * M[i, state]
    
    if prints:
        print("Pr(H" + str(len(v)+1) + " = s" + str(state+1) + " |", v+1, ") =", " + ".join(calc), "=", prob)
        
    return prob

In [11]:
def get_v_plus_one(state, alphas, M, B, v, prints = True):
    N = M.shape[1]
    h_1 = [get_h_plus_one(i, alphas, M, v, prints = False) for i in range(N)]
    
    prob = 0
    calc = []
    
    for i in range(N):
        calc.append(str(h_1[i]) + " * " + str(B[i, state]))
        prob += h_1[i] * B[i, state]
    
    if prints:
        print("Pr(V" + str(len(v)+ 1) + " = x" + str(state + 1), "|", v+1, ") =", " + ".join(calc), "=", prob)

Input data here, for the sequence subtract each state by 1. In this example the actual sequence was 4, 1, 2

In [12]:
M = np.array([[0.8, 0.2], [0.3, 0.7]])
B = np.array([[0.3, 0.4, 0.1, 0.2], [0.2, 0.2, 0.3, 0.3]])
p = np.array([[0.4], [0.6]])
v = np.array([3, 0, 1])

In [13]:
alphas = calculate_alphas(M, p, B, v)

print("\nFinal matrix\n", alphas)

print("\nProbability of sequence", v+1, "=", sum(alphas[:, -1]))

print("\nDistribution of H" + str(len(v)) + ":")

for i in range(len(p)):
    print("Pr(H" + str(len(v)) + " = s" + str(i+1) + " | ", v+1, ") = ", alphas[i, -1]/sum(alphas[:, -1]))
    
print("\nPredicting H" + str(len(v) + 1) + ":")

for i in range(len(p)):
    get_h_plus_one(i, alphas, M, v)
    
print("\nPredicting V" + str(len(v) + 1) + ":")

for i in range(len(v) + 1):
    get_v_plus_one(i, alphas, M, B, v)

t1
alpha 1 = [0.4] * 0.2 = 0.08000000000000002
alpha 2 = [0.6] * 0.3 = 0.18
t2
alpha 1 = 0.3 * [ (0.08000000000000002 * 0.8) + (0.18 * 0.3) ] = 0.03540000000000001
alpha 2 = 0.2 * [ (0.08000000000000002 * 0.2) + (0.18 * 0.7) ] = 0.028400000000000005
t3
alpha 1 = 0.4 * [ (0.03540000000000001 * 0.8) + (0.028400000000000005 * 0.3) ] = 0.014736000000000006
alpha 2 = 0.2 * [ (0.03540000000000001 * 0.2) + (0.028400000000000005 * 0.7) ] = 0.005392000000000001

Final matrix
 [[0.08     0.0354   0.014736]
 [0.18     0.0284   0.005392]]

Probability of sequence [4 1 2] = 0.020128000000000007

Distribution of H3:
Pr(H3 = s1 |  [4 1 2] ) =  0.7321144674085851
Pr(H3 = s2 |  [4 1 2] ) =  0.2678855325914149

Predicting H4:
Pr(H4 = s1 | [4 1 2] ) = 0.7321144674085851 * 0.8 + 0.2678855325914149 * 0.3 = 0.6660572337042926
Pr(H4 = s2 | [4 1 2] ) = 0.7321144674085851 * 0.2 + 0.2678855325914149 * 0.7 = 0.3339427662957074

Predicting V4:
Pr(V4 = x1 | [4 1 2] ) = 0.6660572337042926 * 0.3 + 0.3339427662957074

## Week 2: Beta

In [14]:
def calculate_betas(M, p, B, v):
    N, T = p.shape[0], v.shape[0]
    betas = np.zeros(shape=(N, T))
    
    for i in range(N):
        betas[i, -1] = 1
        
    print("t" + str(T), "initialise all values at 1\n", betas)
    
    def recursive_step(t):
        if abs(t) <= len(v):
            print("t" + str(3-abs(t) + 1))
            for i in range(N):
                beta_i = 0
                strs = []
                for j in range(N):
                    beta_i += betas[j, t + 1] * M[i, j] * B[j][v[t+1]]
                    strs.append("(" + str(betas[j, t + 1]) + " * " + str(M[i, j]) + " * " + str(B[j][v[t+1]]) + ")")
                    
                betas[i, t] = beta_i
                print("beta", i+1, "=", " + ".join(strs), "=", betas[i, t])
                
            recursive_step(t - 1)
    
    recursive_step(-2)
    return betas

Input data here

In [15]:
M = np.array([[0.8, 0.2], [0.3, 0.7]])
B = np.array([[0.3, 0.4, 0.1, 0.2], [0.2, 0.2, 0.3, 0.3]])
p = np.array([[0.4], [0.6]])
v = np.array([3, 0, 1])

betas = calculate_betas(M, p, B, v)
print("\nFinal matrix\n", betas)

t3 initialise all values at 1
 [[0. 0. 1.]
 [0. 0. 1.]]
t2
beta 1 = (1.0 * 0.8 * 0.4) + (1.0 * 0.2 * 0.2) = 0.3600000000000001
beta 2 = (1.0 * 0.3 * 0.4) + (1.0 * 0.7 * 0.2) = 0.26
t1
beta 1 = (0.3600000000000001 * 0.8 * 0.3) + (0.26 * 0.2 * 0.2) = 0.09680000000000002
beta 2 = (0.3600000000000001 * 0.3 * 0.3) + (0.26 * 0.7 * 0.2) = 0.0688

Final matrix
 [[0.0968 0.36   1.    ]
 [0.0688 0.26   1.    ]]


## Week 2: Gamma

In [16]:
def calculate_gamma(alphas, betas):
    N, T = alphas.shape
    gammas = np.zeros(shape=(N, T))
    
    for t in range(T):
        for i in range(N):
            gammas[i, t] = alphas[i, t] * betas[i, t] / np.sum(alphas[:, t] * betas[:, t])
    
    return gammas

Input data here

In [17]:
M = np.array([[0.8, 0.2], [0.3, 0.7]])
B = np.array([[0.3, 0.4, 0.1, 0.2], [0.2, 0.2, 0.3, 0.3]])
p = np.array([[0.4], [0.6]])
v = np.array([3, 0, 1])

In [18]:
print("\nGamma matrix\n", calculate_gamma(calculate_alphas(M, p, B, v), calculate_betas(M, p, B, v)))

t1
alpha 1 = [0.4] * 0.2 = 0.08000000000000002
alpha 2 = [0.6] * 0.3 = 0.18
t2
alpha 1 = 0.3 * [ (0.08000000000000002 * 0.8) + (0.18 * 0.3) ] = 0.03540000000000001
alpha 2 = 0.2 * [ (0.08000000000000002 * 0.2) + (0.18 * 0.7) ] = 0.028400000000000005
t3
alpha 1 = 0.4 * [ (0.03540000000000001 * 0.8) + (0.028400000000000005 * 0.3) ] = 0.014736000000000006
alpha 2 = 0.2 * [ (0.03540000000000001 * 0.2) + (0.028400000000000005 * 0.7) ] = 0.005392000000000001
t3 initialise all values at 1
 [[0. 0. 1.]
 [0. 0. 1.]]
t2
beta 1 = (1.0 * 0.8 * 0.4) + (1.0 * 0.2 * 0.2) = 0.3600000000000001
beta 2 = (1.0 * 0.3 * 0.4) + (1.0 * 0.7 * 0.2) = 0.26
t1
beta 1 = (0.3600000000000001 * 0.8 * 0.3) + (0.26 * 0.2 * 0.2) = 0.09680000000000002
beta 2 = (0.3600000000000001 * 0.3 * 0.3) + (0.26 * 0.7 * 0.2) = 0.0688

Gamma matrix
 [[0.38473768 0.63314785 0.73211447]
 [0.61526232 0.36685215 0.26788553]]


## Week 3: moving average smoothing

In [19]:
def ma(series, span, element_range, weights = None):
    start, stop = element_range
    k = int((span - 1)/2)
    
    if weights is None:
        weights = np.ones((1, span))[0]
    else:
        span = 1
    
    return [(sum(np.array(series[i-k:i+k+1]) * weights))/span for i in range(start, stop+1, 1)]

Remember to subtract one from each element of the range. So in this example we needed to apply to the elements [2, 4]. If you want equal weight just remove the weights argument below e.g ma(series = [4, 2, 3, 5, 1], span = 3, element_range = (1, 3))

In [20]:
ma(series = [4, 2, 3, 5, 1], span = 3, element_range = (1, 3), weights = [0.25, 0.5, 0.25])

[2.75, 3.25, 3.5]

## Week 3: differencing

In [21]:
def difference(series, n):
    if n == 0:
        return series
    
    return difference([series[i+1] - series[i] for i in range(len(series) - 1)], n - 1)

In [22]:
print(difference(series = [4, 2, 3, 5, 1], n = 2))

[3, 1, -6]


## Week 3: box-cox

In [23]:
lamb = 0.5
series = [4, 2, 3, 5, 1]

box_cox = lambda x, l: ((x**l) - 1)/l if 0 < l and l <= 1 else math.log(x)
[box_cox(x, lamb) for x in series]

[2.0, 0.8284271247461903, 1.4641016151377544, 2.4721359549995796, 0.0]

## Week 3: Simple E.M.A

In [24]:
def simple_ema(series, alpha, l0):
    smoothed = [l0]
    
    for i in range(len(series)):
        smoothed.append((alpha * series[i]) + (1 - alpha) * smoothed[i])
    
    return smoothed

In [25]:
simple_ema(series = [4, 2, 3, 5], alpha = 0.5, l0 = 2)

[2, 3.0, 2.5, 2.75, 3.875]

## Week 3: Simple E.M.A tool to predict next given current observation and past prediction

See question 8(a) in the week 3 questions

In [26]:
get_pred = lambda new, past, alpha: alpha * new + ((1 - alpha) * past)

In [27]:
get_pred(5, 4, 0.7)

4.7

## Week 3: double exponential smoothing prediction given a new observation

In [28]:
def double_ema(level, trend, alpha, beta, new_observation, steps):
    new_level = alpha * new_observation + (1 - alpha) * (level + trend)
    new_trend = beta * (new_level - level) + (1 - beta) * trend
    return (new_level, new_trend, new_level + (steps * new_trend))

Output order: level, trend, prediction

In [29]:
double_ema(level = 3, trend = 1, alpha = 0.7, beta = 0.8, new_observation = 5, steps = 2)

(4.7, 1.5600000000000003, 7.82)

## Week 3: triple ema prediction for next step

In [30]:
def double_ema(level, trend, s_t_m, s_t_m_1, alpha, beta, gamma, new_observation):
    new_level = alpha * (new_observation - s_t_m) + (1 - alpha) * (level + trend)
    new_trend = beta * (new_level - level) + (1 - beta) * trend
    new_seasonality = gamma * (new_observation - level - trend) + (1 - gamma) * s_t_m
    prediction = new_level + new_trend + s_t_m_1
    
    return (new_level, new_trend, new_seasonality, prediction)

Output order: level, trend, seasonality, prediction

In [31]:
double_ema(level = 3, trend = 1, s_t_m = 2, s_t_m_1 = 1, alpha = 0.7, beta = 0.8, gamma = 0.9, new_observation = 5)

(3.3, 0.43999999999999984, 1.1, 4.74)

## Week 3: Calculate sample autocovariance and autocorrelation functions for weakly stationary processes

In [32]:
def get_autos(series, lag):
    gamma = lambda h, seq, mean, T: (1/T) * sum([(seq[i] - mean) * (seq[i + h] - mean) for i in range(T - h)])
    
    T = len(series)
    mean = np.mean(series)
    
    lagh = gamma(lag, series, mean, T)
    lag0 = gamma(0, series, mean, T)
    
    return (lagh, (lagh / lag0))

Output order: gamma hat, p hat aka autocovariance and then autocorrelation

In [33]:
get_autos(series = [4, 2, 3, 5, 1], lag = 0)

(2.0, 1.0)

## Week 4: halving algorithm

In [86]:
def halving_algorithm(true_outcomes, experts_predictions):
    E, N = experts_predictions.shape
    whitelist = np.array(range(E))
    
    for i in range(N):
        print("Prediction for time " + str(i+1), stats.mode(experts_predictions[whitelist, i])[0])
        
        new_whitelist = np.array([j for j in whitelist if experts_predictions[j, i] == true_outcomes[i]])
        
        print("Experts", np.setdiff1d(whitelist, new_whitelist) + 1, "were wrong and are now in the blacklist\n")
        
        whitelist = new_whitelist
        

In [87]:
halving_algorithm(true_outcomes = [0, 0, 0, 1, 1], experts_predictions = np.array([[1, 0, 0, 0, 0],
                                                                                     [0, 0, 0, 1, 1],
                                                                                     [0, 1, 1, 1, 0]]))

Prediction for time 1 [0]
Experts [1] were wrong and are now in the blacklist

Prediction for time 2 [0]
Experts [3] were wrong and are now in the blacklist

Prediction for time 3 [0]
Experts [] were wrong and are now in the blacklist

Prediction for time 4 [1]
Experts [] were wrong and are now in the blacklist

Prediction for time 5 [1]
Experts [] were wrong and are now in the blacklist



## Week 4: weighted-majority algorithm

In [124]:
def weighted_majority(true_outcomes, experts_predictions, beta):
    E, N = experts_predictions.shape
    weights = np.ones((1, E))[0]
    
    for i in range(N):
        print("Time", i+1)
        print("Weights", weights)
        # Prediction
        d = defaultdict(int)
        
        for j in range(E):
            d[experts_predictions[j, i]] += weights[j]
        
        #print({k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse = True)})
        print("Prediction:", list({k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse = True)})[0], "\n")
        
        # Updating weights
        losses = [0 if experts_predictions[j, i] == true_outcomes[i] else 1 for j in range(E)]
        updates = np.array([beta**x for x in losses])
        weights *= updates

In [125]:
weighted_majority(true_outcomes = [0, 1, 0, 1, 1], experts_predictions = np.array([[1, 0, 0, 0, 0],
                                                                                 [0, 0, 0, 1, 1],
                                                                                 [0, 1, 1, 1, 0]]), beta = 0.5)

Time 1
Weights [1. 1. 1.]
Prediction: 0 

Time 2
Weights [0.5 1.  1. ]
Prediction: 0 

Time 3
Weights [0.25 0.5  1.  ]
Prediction: 1 

Time 4
Weights [0.25 0.5  0.5 ]
Prediction: 1 

Time 5
Weights [0.125 0.5   0.5  ]
Prediction: 0 

